# Multiprocessing and Distributed Computing

**What**
* **single Julia process -> multiple Julia processes** that coordinate to perform certain computations

**Why**
* **Scaling things up**: run computations on multiple CPU cores, potentially even on different machines, e.g. nodes of a supercomputer or a local cluster of desktop machines.
* Effectively increase your memory: process a large dataset, which wouldn't fit into local memory, in parallel across multiple machines with separate dedicated RAM.
* (Typically avoids some multithreading / shared memory computing issues: e.g. multiple GCs.)

**Julia provides two fundamental implementations and paradigms**
* Julia's built-in [Distributed.jl](https://docs.julialang.org/en/v1/stdlib/Distributed/) standard library
* [Message Passing Interface (MPI)](https://www.mpi-forum.org/) through [MPI.jl](https://github.com/JuliaParallel/MPI.jl)

 

## Distributed.jl (standard library) vs MPI

**Distributed.jl**
* convenient for **"ad-hoc" distributed computing** (e.g. data processing)
* intuitive **master-worker model** (often naturally aligns with the structure of scientific computations)
* **interactivity**, e.g. in a REPL / in Jupyter
* built-in, no external setup necessary
* higher overhead than MPI and doesn't scale as well (by default doesn't utilizie Infiniband/OmniPath)

**MPI**
* **de-facto industry standard** for massively parallel computing, e.g. large scale distributed computing
* **known to scale well** up to thousands of compute nodes
* Single Program Multiple Data (SPMD) programming model can be more challenging (at least at first)
* **No (or very poor) interactivity** (see [MPIClusterManager.jl](https://github.com/JuliaParallel/MPIClusterManagers.jl))


The focus of this notebook is on **Distributed.jl** (MPI → later).

## Distributed.jl (standard library)

Julia's Distributed.jl follows a master-worker paradigm for its native distributed parallelism: **One master process coordinates all the worker processes, which perform the actual computations.**

In [ ]:
using Distributed

In [ ]:
nprocs()

In [ ]:
nworkers() # the master is considered a worker as long as there are no real workers

To increase the number of workers, i.e. Julia processes, from within a Julia session we can dynamically call **`addprocs`**.

Alternatively, when starting Julia from the command line, one can use the `-p` option up front. Example,

```
julia -p 4
```

will start Julia with 5 processes, 1 master and 4 workers.

In [ ]:
addprocs(4)

Every process has a Julia internal `pid` (process id). The master is always 1. You can get the workers pids from `workers()`.

In [ ]:
workers()

Note that the 4 worker's pids aren't necessarily 2, 3, 4 and 5 and one shouldn't rely on those literal values. Let's remove the processes and add them once more.

In [ ]:
rmprocs(workers())

In [ ]:
nworkers() # only the master is left

In [ ]:
addprocs(4)

In [ ]:
workers()

### One master to rule them all - `@spawn`, `@spawnat`, `@fetch`, `@fetchfrom`, `@everywhere`...

To execute commands and start computations on workers we can use the following macros

* `@spawn`: run a command or a code block on any worker and return a `Future` (a wrapped `Task`) to it's result.
* `@spawnat`: same as `@spawn` but one can choose a specific worker by providing its pid.

Note the conceptual similarity between `Threads.@spawn` (task → thread) and `Distributed.@spawn` (task → process) and also `@async`.

In [ ]:
@spawn 3+3

In [ ]:
result = @spawn 3+3

In [ ]:
fetch(result)

Because the combination of spawning at fetching is so common, there is `@fetch` (blocking) which combines them.

In [ ]:
@fetch 3+3

In [ ]:
@fetch rand(3,3)

Which worker did the work?

In [ ]:
@fetch begin
    println(myid());
    3+3
end

Using `@spawnat` and `@fetchfrom` we can delegate the work to a specific worker.

In [ ]:
@fetchfrom 7 begin
    println(myid());
    3+3
end

We can use `@sync` as a blocker to wait for all workers to complete their tasks.

In [ ]:
@sync begin
    pids = workers()
    @spawn (sleep(2); println("Today is reverse day!"))
    @spawn (sleep(1); println(" Stuttgart!"))
    @spawn println("Hello")
end;
println("Done!")

Ok, now that we understood all that, let's delegate a *complicated* calculation

In [ ]:
using Random

function complicated_calculation()
    sleep(1) # so complex that it takes a long time :)
    randexp(5)
end

@fetch complicated_calculation()

What happened?

**Every worker is a separate Julia process.** (Think of having multiple Julia REPLs open at once.)

We only defined `complicated_calculation()` on the master process. The function doesn't exist on any of the workers yet.

The macro `@everywhere` allows us to perform steps on all processes (master and worker). This is particularly useful for loading packages and functions definitions etc.

In [ ]:
@everywhere begin # execute this block on all workers
    using Random
    
    function complicated_calculation()
        sleep(1)
        randexp(5) # lives in Random
    end
end

In [ ]:
@fetch complicated_calculation()

### Data movement

There is a crucial difference between the following two pieces of code. Can you guess what it is? (without reading on 😉)

In [ ]:
function method1()
    A = rand(100,100)
    B = rand(100,100)
    C = @fetch A^2 * B^2
end

In [ ]:
function method2()
    C = @fetch rand(100,100)^2 * rand(100,100)^2
end

Let's benchmark them.

In [ ]:
using BenchmarkTools
@btime method1();
@btime method2();

Method 1 is slower, because `A` and `B` are created on the master process, transferred to a worker, and squared and multiplied on the worker process before the result is finally transferred back to the master.

Method 2, on the other hand, creates, squares, and multiplies the random matrix all on the work process and only submits the result to the master.

Hence, `method1` is **transferring 3x as much data** between the master and the worker!

In this toy example, it's rather easy to identify the faster method.

In a real program, however, understanding data movement does require more thought and likely some measurement.

For example, if the first process needs matrix `A` in a follow-up computation then the first method might be better in this case. Or, if computing `A` is expensive and only the current process has it, then moving it to another process might be unavoidable.

#### Computer latency at a human scale

To understand why thinking about data is important it's instructive to look at the time scales involved in data access.

<img src="./imgs/latency_human_scales.png" width=900px>

(taken from https://www.prowesscorp.com/computer-latency-at-a-human-scale/)

**Efficient data movement is crucial for efficient distributed computing!** (!)

Note that there are more tools for explicit data movement like `Channels`/`RemoteChannels` and [ParallelDataTransfer.jl](https://github.com/ChrisRackauckas/ParallelDataTransfer.jl/). For the sake of brevity, we will not cover them here.

#### Avoid globals (once more)

In [ ]:
myglobal = 4

In [ ]:
function whohas(s::String)
    @everywhere begin
        var = Symbol($s)
        if isdefined(Main, var)
            println("$var exists.")
        else
            println("Doesn't exist.")
        end
    end
    nothing
end

In [ ]:
whohas("myglobal")

In [ ]:
@fetchfrom 6 myglobal+2

In [ ]:
whohas("myglobal")

Globals get copied to workers and continue to exist as globals even after the call.

This could lead to **memory accumulation** if many globals are used (just as it would in a single Julia session). → avoid them as much as possible

## High-level tools: `@distributed` and `pmap`

So far we have seen some of the fundamental building blocks for distributed computing with Distributed.jl. However, in practice, one wants to think as little as possible about how to distribute the work and explicitly spawn tasks.

Fortunately, many useful parallel computations do not require (much) data movement at all. A common example is a direct Monte Carlo simulation, where multiple processes can handle independent simulation trials simultaneously. (→ **montecarlo_pi** exercise)

Julia provides **high-level convenience** tools to
 * distribute `map` $\quad$ → $\quad$ [`pmap`](https://docs.julialang.org/en/v1/stdlib/Distributed/#Distributed.pmap)
 * distribute loops $\quad$ → $\quad$ [`@distributed`](https://docs.julialang.org/en/v1/stdlib/Distributed/#Distributed.@distributed)

### Parallel map: `pmap`

Very often, one simply wants to parallize a `map` operation where a function `f` is applied to all elements of a collection. This is a typical instance of **data parallelism**, which covers a vast class of compute-intensive programs.

In [ ]:
map(x->x^2, 1:10)

Such a pattern can be parallelized in Julia via the high-level function `pmap` ("parallel map").

**Remark:** No (non-local) data structure mutation in the function `f`!

#### Example: Singular values of multiple matrices

In [ ]:
using Distributed, BenchmarkTools; rmprocs(workers()); addprocs(4); nworkers() # clean reset :)

In [ ]:
@everywhere using LinearAlgebra

M = Matrix{Float64}[rand(200,200) for i = 1:10];

In [ ]:
svdvals(rand(2,2))

In [ ]:
map(svdvals, M)

In [ ]:
pmap(svdvals, M)

Let's check that this indeed utilized multiple workers.

In [ ]:
pmap(M) do m
    println(myid());
    svdvals(m)
end;

In [ ]:
@btime map($svdvals, $M);
@btime pmap($svdvals, $M);

#### Distributed loops (`@distributed`)

In [ ]:
using Distributed, BenchmarkTools; rmprocs(workers()); addprocs(4); nworkers() # clean reset :)

#### Example: Reduction

Task: Counting heads in a series of coin tosses.

In [ ]:
function count_heads_loop(n)
    c = 0
    for i = 1:n
        c += rand((0,1))
    end
    return c
end

N = 200_000_000
@btime count_heads_loop($N);

Note that these kinds of computations are called **reductions** (with `+` being the **reducer function**).

In [ ]:
count_heads_reduce(n) = mapreduce(i -> rand((0,1)), +, 1:n)
@btime count_heads_reduce($N)

We can parallelize this reduction using
* `@distributed (reducer function) for ...`.

Note that this has **blocking** character and returns the result.

In [ ]:
function count_heads_distributed_loop(n)
    c = @distributed (+) for i in 1:n
        rand((0,1))
    end
    return c
end

In [ ]:
@btime count_heads_distributed_loop($N);

The distributed version is about **4x faster**, which is all we could hope for.

With `@distributed` the work is **evenly distributed** between the workers.

In [ ]:
function count_heads_distributed_verbose(n)
    c = @distributed (+) for i in 1:n
        x = rand((0,1))
        println(x);
        x
    end
    c
end

count_heads_distributed_verbose(8);

#### A common mistake

In [ ]:
function g(n)
    a = 0
    @distributed (+) for i in 1:n
        a += 1
    end
    a
end

a = g(10);

What do you expect the value of `a` to be?

In [ ]:
a

#### Example: Array mutation

Apart from `@distributed (reducer) ...` there also is a `@distributed for ...` form. The latter is **non-blocking** and returns a `Task`. (You can think of it as a distributed version of `@spawn` for all the iterations.)

However, since the loop body will be executed on different processes, one must be careful to operate on **data structures that are available on all processes** (similar to the mistake highlighted above).

In [ ]:
function square_broken()
    A = collect(1:10)
    @sync @distributed for i in eachindex(A)
        A[i] = A[i]^2
    end
    return A
end

In [ ]:
square_broken()

To actually make all processes operate on the same array, one can use a `SharedArray`. For this to work, the **processes need to live on the same host**.

In [ ]:
@everywhere using SharedArrays # must be loaded everywhere

In [ ]:
A = rand(3,2)

In [ ]:
S = SharedArray(A)

In [ ]:
function square!(X)
    for i in eachindex(X)
        sleep(0.001) # mimic some computational cost
        X[i] = X[i]^2
    end
end

function square_distributed!(X)
    @sync @distributed for i in eachindex(X)
        sleep(0.001) # mimic some computational cost
        X[i] = X[i]^2
    end
end

A = rand(10,10)
S = SharedArray(A)

@btime square!($A);
@btime square_distributed!($S);

Note that the functions `square!` and `square_distributed!` could have also been written with `map` and `pmap`, respectively. (data parallelism)

### `@distributed` vs `pmap`: When to choose which?

Julia's `pmap` is designed for the case where

* one wants to apply **a function to a collection**,
* one needs **load-balancing** (e.g. workload is non-uniform), and/or
* each function call does enough work to amortize the overhead. 

On the other hand, `@distributed` is good for

* **reductions** (can't be written as `map`/`pmap`),
* loops where each iteration **takes about the same time** (uniform workload), and/or
* loops where the workload of each iteration is (very) small.

## High-level array abstractions: [DistributedArrays.jl](https://github.com/JuliaParallel/DistributedArrays.jl)

In a `DArray`, each process has local access to just a chunk of the data, and no two processes share the same chunk. Processes can be on different hosts.

Distributed arrays are for example useful if

* Expensive calculations should be performed in parallel on parts of the array on different hosts.
* The data doesn't fit into the local machines memory (i.e. loading big files in parallel).

In [ ]:
using Distributed, BenchmarkTools; rmprocs(workers());

In [ ]:
# make sure that all workers use the same Julia environment
addprocs(4; exeflags="--project")

In [ ]:
# check
@everywhere @show Base.active_project()

In [ ]:
@everywhere using DistributedArrays, LinearAlgebra

In [ ]:
M = Matrix{Float64}[rand(200,200) for i = 1:10];

In [ ]:
D = distribute(M)

Which workers hold parts of D?

In [ ]:
procs(D)

In [ ]:
workers()

Which parts do they hold?

In [ ]:
localpart(D) # the master doesn't hold anything

In [ ]:
# Which parts do they hold?
for p in workers()
    println(@fetchfrom p DistributedArrays.localindices(D))
end

In [ ]:
@btime map($svdvals, $M);
@btime map($svdvals, $D);

In [ ]:
@btime pmap($svdvals, $M);

## *Actual* distributed computing with Distributed.jl (some comments)

### Creating workers on other machines

So far we have worked with multiple process on the same system, because we simply used `addprocs(::Integer)`. To put worker processes on other machines, e.g. nodes of a cluster, we need to modify the initial `addprocs` call appropriately.

In Julia, starting worker processes is handled by [ClusterManagers](https://docs.julialang.org/en/v1/manual/distributed-computing/#ClusterManagers).

* The default one is `LocalManager`. It is automatically used when running `addprocs(i::Integer)` and we have implicitly used it already.
* Another important one is `SSHManager`. It is automatically used when running `addprocs(hostnames::Array)`, e.g. `addprocs(["node123", "node456"])`. The only requirement is a **passwordless ssh access** to all specified hosts.
* Cluster managers for SLURM, PBS, and others are provided in [ClusterManagers.jl](https://github.com/JuliaParallel/ClusterManagers.jl). For SLURM, this will make `addprocs` use `srun` under the hood.

*Example*

```julia
using Distributed

addprocs(["node123", "node123"])

@everywhere println(gethostname())
```

One can also start multiple processes on different machines:
```julia
addprocs([("node123", 2), ("node456", 3)]) # starts 2 workers on node123 and 3 workers on node456

# Use :auto to start as many processes as CPU threads are available
```

**Be aware of different paths:**
* By default, `addprocs` expects to find the julia executable on the remote machines under the same path as on the host (master).
* It will also try to `cd` to the same folder (set the working directory).


As you can see from `?addprocs`, `addprocs` takes a bunch of keyword arguments, two of which are of particular importance in this regard:

* `dir`: working directory for the worker processes
* `exename`: path to julia executable (potentially augmented with pre-commands) for the worker processes

In [ ]:
# cleanup
rmprocs(workers())